# Here we merge our cleaned data

## Importing libraries

In [1]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import date

## Loading the Datasets

In [2]:
df_industries = pd.read_csv('Prepared Frames/industry_data.csv')
df_companies = pd.read_csv('Prepared Frames/company_data.csv')
df_balance = pd.read_csv('Prepared Frames/balance_data.csv')
df_cashflow = pd.read_csv('Prepared Frames/cashflow_data.csv')
df_income = pd.read_csv('Prepared Frames/income_data.csv')

In [3]:
display(df_industries.head(1))
display(df_companies.head(1))
display(df_balance.head(1))
display(df_cashflow.head(1))

,IndustryId,Sector,Industry
0,100001,Industrials,Industrial Products


,Ticker,SimFinId,Company Name,IndustryId,tokens,current_search
0,A,45846,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0


,Ticker,Fiscal Year,Fiscal Period,Report Date,Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Total Current Assets,"Property, Plant & Equipment, Net",Other Long Term Assets,Total Noncurrent Assets,Total Assets,Payables & Accruals,Total Current Liabilities,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Retained Earnings,Total Equity
0,A,2017,Q1,2017-01-31,326000000.0,2.241000e+09,653000000.0,3.635000e+09,653000000.0,3.451000e+09,4.237000e+09,7872000000,268000000.0,1089000000,2.483000e+09,3572000000,5.239000e+09,-453000000.0,4.300000e+09


,Ticker,Fiscal Year,Fiscal Period,Net Income/Starting Line,Depreciation & Amortization,Non-Cash Items,Change in Working Capital,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
0,A,2017,Q1,168000000.0,55000000.0,45000000.0,-152000000.0,116000000.0,-32000000.0,-69000000.0,-101000000.0,89000000.0,-93000000.0,-58000000.0,-48000000


## Merging

#### Cashflow and Balance Sheet
We will have to merge the cashflow and balance sheet data on the Ticker, Fiscal Year and Fiscal Period, to get the related reports
together.

In [4]:
print(df_balance.shape)
print(df_cashflow.shape)

(37222, 19)
(37499, 15)


In [5]:
df_statements = pd.merge(left = df_balance,
                                 right = df_cashflow,
                                 # We use outer to not loose Information, we will have to get rid of some NaNs later
                                 how = 'outer', 
                                 left_on = ['Ticker','Fiscal Year','Fiscal Period'], 
                                 right_on= ['Ticker','Fiscal Year','Fiscal Period'])
df_statements.shape

(37499, 31)

In [6]:
# df_statements.isna().sum()
# We created some rows with NaNs, we cannot get any additional information at this point and have to drop them.
df_statements = df_statements[df_statements['Report Date'].isna() == False]
df_statements.shape

(37222, 31)

#### Income data

In [7]:
print(df_statements.shape)
print(df_income.shape)

(37222, 31)
(37809, 19)


In [8]:
df_statements = pd.merge(left = df_balance,
                                 right = df_income,
                                 # We use outer to not loose Information, we will have to get rid of some NaNs later
                                 how = 'outer', 
                                 left_on = ['Ticker','Fiscal Year','Fiscal Period'], 
                                 right_on= ['Ticker','Fiscal Year','Fiscal Period'])
df_statements.shape

(37829, 35)

## Names and Industries
Now we will add the information from our companies dataframe, especially the name is needed, to assign the ratings later

In [9]:
df_main = pd.merge(left = df_companies,
                                 right = df_statements,
                                 # We use outer to not loose Information, we will have to get rid of some NaNs later
                                 how = 'right', 
                                 left_on = ['Ticker'], 
                                 right_on= ['Ticker'])
df_main.shape

(37829, 40)

In [10]:
# We will also add the Industry names, since it makes the analysis better understandable than the Industry IDs
df_main = pd.merge(left = df_main,
                                 right = df_industries,
                                 # We use outer to not loose Information, we will have to get rid of some NaNs later
                                 how = 'left', 
                                 left_on = ['IndustryId'], 
                                 right_on= ['IndustryId'])
df_main.shape

(37829, 42)

In [11]:
# df_main.isna().sum()
# No more NaNs were created

df_main.head(50)

,Ticker,SimFinId,Company Name,IndustryId,tokens,current_search,Fiscal Year,Fiscal Period,Report Date,Shares (Diluted),...,"Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common),Sector,Industry
0,A,45846.0,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0.0,2017,Q1,2017-01-31,326000000.0,...,193000000.0,0.0,193000000.0,-25000000.0,168000000.0,0.0,168000000.0,168000000.0,Healthcare,Medical Diagnostics & Research
1,A,45846.0,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0.0,2017,Q2,2017-04-30,325000000.0,...,191000000.0,0.0,191000000.0,-27000000.0,164000000.0,0.0,164000000.0,164000000.0,Healthcare,Medical Diagnostics & Research
2,A,45846.0,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0.0,2017,Q3,2017-07-31,326000000.0,...,193000000.0,0.0,193000000.0,-18000000.0,175000000.0,0.0,175000000.0,175000000.0,Healthcare,Medical Diagnostics & Research
3,A,45846.0,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0.0,2017,Q4,2017-10-31,327000000.0,...,226000000.0,0.0,226000000.0,-49000000.0,177000000.0,0.0,177000000.0,177000000.0,Healthcare,Medical Diagnostics & Research
4,A,45846.0,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0.0,2018,Q1,2018-01-31,323000000.0,...,233000000.0,0.0,233000000.0,-553000000.0,-320000000.0,0.0,-320000000.0,-320000000.0,Healthcare,Medical Diagnostics & Research
5,A,45846.0,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0.0,2018,Q2,2018-04-30,326000000.0,...,227000000.0,0.0,227000000.0,-22000000.0,205000000.0,0.0,205000000.0,205000000.0,Healthcare,Medical Diagnostics & Research
6,A,45846.0,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0.0,2018,Q3,2018-07-31,324000000.0,...,242000000.0,0.0,242000000.0,-6000000.0,236000000.0,0.0,236000000.0,236000000.0,Healthcare,Medical Diagnostics & Research
7,A,45846.0,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0.0,2018,Q4,2018-10-31,327000000.0,...,244000000.0,0.0,244000000.0,-49000000.0,195000000.0,0.0,195000000.0,195000000.0,Healthcare,Medical Diagnostics & Research
8,A,45846.0,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0.0,2019,Q1,2019-01-31,322000000.0,...,248000000.0,0.0,248000000.0,256000000.0,504000000.0,0.0,504000000.0,504000000.0,Healthcare,Medical Diagnostics & Research
9,A,45846.0,AGILENT TECHNOLOGIES INC,106001.0,"['AGILENT', 'TECHNOLOGIES', 'INC']",0.0,2019,Q2,2019-04-30,321000000.0,...,218000000.0,0.0,218000000.0,-36000000.0,182000000.0,0.0,182000000.0,182000000.0,Healthcare,Medical Diagnostics & Research


## Saving the concatenated Dataframe

In [12]:
df_main.to_csv('Prepared Frames/main_data.csv', index=False)